In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey, select
from sqlalchemy.sql import and_, or_, not_
import re
#from collections import Counter
import uuid 
from sklearn.model_selection import train_test_split

In [3]:
engine = create_engine('sqlite:///HoopStat.db')
conn = engine.connect()
metadata = MetaData(bind=None)
games = Table('games', metadata, autoload = True, autoload_with = engine)
events = Table('events', metadata, autoload = True, autoload_with = engine)

In [4]:
stmt = select([games.c.id]).where(games.columns.isProcessed == True)
results = conn.execute(stmt).fetchall()
gameIDs = [value for value, in results]
len(gameIDs)

5025

In [5]:
def cleanAction(x):
    x = re.sub('^\d+', '', x).lstrip()
    x = re.sub('\d+$', '', x).rstrip()
    x = re.sub(' by$', '', x).rstrip()
    return x
def cleanPeriod(x):
    x = int(''.join(filter(str.isdigit, x)))
    return x
def cleanPlayer(x):
    x = x.lstrip().rstrip()
    return x
def getStarters(df):
    periodStart = pd.to_timedelta('00:20:00')
    periodEnd = pd.to_timedelta('00:00:00')
    lineups2 = df[df.action.isin(['enters the game','goes to the bench'])
                 ][['player','action','time','period','team']]
    #lineups2['time'] = lineups2['time'].apply(pd.to_datetime) + pd.DateOffset(lineups2['period'])
    linePV = pd.pivot_table(lineups2,index=['player','team'],columns='action',values='time',aggfunc=np.max).reset_index()
    linePV['enters the game'] = linePV['enters the game'].fillna(periodStart)
    linePV['goes to the bench'] = linePV['goes to the bench'].fillna(periodEnd)
    
    starters = linePV[
        (
            (linePV['goes to the bench'] > linePV['enters the game'])
        )
        |
        (
            (linePV['enters the game'] == '00:20:00')
        )
                     ][['team','player']]
    return list(starters[starters['team']=='Home']['player']),list(starters[starters['team']=='Away']['player'])
    #linePV.sort_values(by=['period','team','goes to the bench'])
    #lineups2.sort_values(by=['time'])

def groupToList(df,k):
         keys, values = df.sort_values(k).values.T
         ukeys, index = np.unique(keys, True)
         arrays = np.split(values, index[1:])
         df2 = pd.DataFrame({k:ukeys, 'b':[list(a) for a in arrays]})
         return df2

In [6]:
actValMap = {
'Assist':0,
'Block':0,
'Foul':0,
'Steal':0,
'Technicaloul':0,
'Turnover':0,
'deadball rebound':0,
'defensive rebound':0,
'enters the game':0,
'goes to the bench':0,
'made 2-pt field goal':2,
'made 3-pt jump shot':3,
'made dunk':2,
'made free throw':1,
'made jump shot':2,
'made layup':2,
'made tip-in':2,
'missed 2-pt field goal':0,
'missed 3-pt jump shot':0,
'missed dunk':0,
'missed free throw':0,
'missed jump shot':0,
#missed jump shot3missed 3-pt jump shot2missed 2-pt field goalmissed layupmissed free throwmissed dunkmissed tip-in}	123
'missed layup':0,
'missed tip-in':0,
'offensive rebound':0,
'pointswrong basket by defense':-2,
#score=2text=made jump shot3score=3text=made 3-pt jump shot2score=2text=made 2-pt field goalscore=2text=made layupscore=1text=made free throwscore=2text=made dunkscore=2text=made tip-in	108
'will be starting':0
}

In [15]:
stmt = "SELECT * FROM events WHERE gameid = '" + gameIDs[0] + "'"
n1 = 0
    
df = pd.read_sql_query(stmt,engine)
df['scoreHome'] = df['scoreHome'].map(int)
df['scoreAway'] = df['scoreAway'].map(int)

df['action'] = df['action'].apply(cleanAction)
df['period'] = df['period'].apply(cleanPeriod).apply(int)
df['player'] = df['player'].apply(cleanPlayer)
df['duration'] = df['duration'].apply(int)

df['actionValue'] = df['action'].map(actValMap).map(int,na_action='ignore')

df['time'] = pd.to_timedelta('00:'+df['time'])
df.loc[df['period'] <= 2,'time'] = pd.to_timedelta('00:20:00') - df.loc[df['period'] <= 2,'time']
df.loc[df['period'] > 2,'time'] = pd.to_timedelta('00:05:00') - df.loc[df['period'] > 2,'time']
df['time'] = df['time'] + pd.to_timedelta(df['period']-1, unit='d') #treat periods as days, units of time

df['seqNo'] = df['time'].ne(df['time'].shift()).cumsum()

df = df.sort_values(by=['time'],ascending=True)

conditions = [
    (df['actionValue'] == 1),
    (df['actionValue'] == 2),
    (df['actionValue'] == 3),
    (df['action'].str.contains('missed') & df['action'].str.contains('3')),
    (df['action'].str.contains('missed') & ~df['action'].str.contains('3') & ~df['action'].str.contains('free')),
    (df['action'].str.contains('missed') & ~df['action'].str.contains('3') & df['action'].str.contains('free'))
]
choices = ['FTM', 'FG2', 'FG3','3PA','2PA','FTA']

df['action_edit1'] = np.select(conditions, choices, default=df['action'])

df['psnChg'] = False

df['playScore'] = df['time'].map(df.groupby("time")['actionValue'].sum())

df['homePS'] = df[df['team']]

#df[df['action_edit1']==0]['action_edit1'] = df[df['action_edit1']==0]['action']
HLU = []
ALU = []
HLU, ALU = getStarters(df)
LUdict = {'Home':{uuid.uuid1():HLU}
         ,'Away':{uuid.uuid1():ALU}
         }

#for i in 
df.head(40)

,id,gameid,time,action,scoreHome,scoreAway,team,duration,player,period,playScore,actionValue,seqNo,action_edit1,psnChg,clockUse
0,20200101_k6nzprd13627661,20200101_k6nz,00:00:13,made 3-pt jump shot,3,0,Home,0,"DAVIS,REGGIE",1,3.0,3.0,1,FG3,False,Early
1,20200101_k6nzprd19434771,20200101_k6nz,00:00:25,missed 3-pt jump shot,3,0,Away,12,"FERGUSON,CAMDEN",1,0.0,0.0,2,3PA,False,Early-Mid
2,20200101_k6nzprd14750351,20200101_k6nz,00:00:30,defensive rebound,3,0,Home,5,"GREENE,CHANCE",1,0.0,0.0,3,defensive rebound,False,Early
3,20200101_k6nzprd197171,20200101_k6nz,00:00:31,Turnover,3,0,Home,1,"LANG,CODY",1,0.0,0.0,4,Turnover,False,Early
4,20200101_k6nzprd18360751,20200101_k6nz,00:00:45,Turnover,3,0,Away,14,"BRYANT,TREY",1,0.0,0.0,5,Turnover,False,Mid
5,20200101_k6nzprd13571151,20200101_k6nz,00:00:45,Steal,3,0,Home,0,"DAVIS,REGGIE",1,0.0,0.0,5,Steal,False,Early
6,20200101_k6nzprd13686711,20200101_k6nz,00:00:53,Assist,5,0,Home,8,"GREENE,CHANCE",1,2.0,0.0,6,Assist,False,Early-Mid
7,20200101_k6nzprd14919151,20200101_k6nz,00:00:53,made layup,5,0,Home,0,"LANG,CODY",1,2.0,2.0,6,FG2,False,Early
8,20200101_k6nzprd19217961,20200101_k6nz,00:01:13,Foul,5,0,Home,20,"GREENE,CHANCE",1,0.0,0.0,7,Foul,False,Late-Mid
9,20200101_k6nzprd12641601,20200101_k6nz,00:01:28,Foul,5,2,Home,15,"DAVIS,REGGIE",1,3.0,0.0,8,Foul,False,Mid


In [ ]:
stmt = "SELECT * FROM events"
conn.execute(stmt).fetchall()

In [ ]:
#view all actions and frequency
#"""
stmt = "\
SELECT \
action\
,COUNT(1) \
FROM events \
WHERE action NOT LIKE '%time%out%'\
GROUP BY \
action \
ORDER BY 2 desc"

act = pd.read_sql_query(stmt,engine)
#"""

act['action'] = df['action'].apply(cleanAction)

df['actionValue'] = df['action'].map(actValMap)

df = df.groupby('action').sum()

df.head(50)

In [ ]:
stmt = "SELECT DISTINCT \
e.gameid\
,e.period\
,e.action AS action1 \
,e2.action AS action2 \
,e.team AS team1 \
,e2.team AS team2 \
FROM events AS e \
INNER JOIN events AS e2 \
ON e.gameid = e2.gameid \
AND e.period = e2.period \
AND e.time = e2.time \
AND e.action > e2.action\
 WHERE e.gameid = '" + gameIDs[0] + "'" 


#print(stmt)
actdf = pd.read_sql_query(stmt,engine)
actdf['action1'] = actdf.action1.apply(cleanAction)
actdf['action2'] = actdf.action2.apply(cleanAction)

actdf.head(25)

In [20]:
metricDF1 = df.copy()
metricDF1.loc[metricDF1['team']=='Away','actionValue'] = metricDF1.loc[metricDF1['team']=='Away','actionValue'] * -1
metricDF1 = metricDF1.set_index('team').groupby('seqNo', as_index=False).agg({
    'action':dict
    , 'actionValue':np.sum
    , 'duration':np.sum
}
)
metricDF1['n_1'] = metricDF1['actionValue'].shift(-1)

metricDF1['clockUse'] = pd.cut(metricDF1['duration'],5,labels = ['Early','Early-Mid','Mid','Late-Mid','Late'])

In [17]:
df[df.action.str.contains('rebound')]

,id,gameid,time,action,scoreHome,scoreAway,team,duration,player,period,playScore,actionValue,seqNo,action_edit1,psnChg,clockUse
2,20200101_k6nzprd14750351,20200101_k6nz,0 days 00:00:30,defensive rebound,3,0,Home,5,"GREENE,CHANCE",1,0.0,0.0,3,defensive rebound,False,Early
17,20200101_k6nzprd17250771,20200101_k6nz,0 days 00:02:21,defensive rebound,10,5,Home,4,"LANG,CODY",1,0.0,0.0,13,defensive rebound,False,Early
19,20200101_k6nzprd15029771,20200101_k6nz,0 days 00:02:31,defensive rebound,10,5,Away,6,"BURWELL,JOHN",1,0.0,0.0,15,defensive rebound,False,Early
23,20200101_k6nzprd18703861,20200101_k6nz,0 days 00:03:04,defensive rebound,10,8,Away,4,"BRYANT,TREY",1,0.0,0.0,18,defensive rebound,False,Early
25,20200101_k6nzprd19613851,20200101_k6nz,0 days 00:03:14,defensive rebound,10,8,Home,3,"THACKER,TC",1,0.0,0.0,20,defensive rebound,False,Early
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
535,20200101_k6nzprd39331301,20200101_k6nz,2 days 00:00:28,offensive rebound,72,71,Home,5,"LANG,CODY",3,1.0,0.0,251,offensive rebound,False,Early
544,20200101_k6nzprd32081471,20200101_k6nz,2 days 00:00:51,defensive rebound,73,71,Home,5,"LANG,CODY",3,0.0,0.0,253,defensive rebound,False,Early
553,20200101_k6nzprd31446971,20200101_k6nz,2 days 00:02:36,defensive rebound,77,77,Away,3,"BURWELL,JOHN",3,0.0,0.0,260,defensive rebound,False,Early
558,20200101_k6nzprd32199621,20200101_k6nz,2 days 00:03:33,defensive rebound,77,79,Away,4,"BRYANT,TREY",3,0.0,0.0,264,defensive rebound,False,Early


In [ ]:
#stmt = "DELETE FROM events"
#conn.execute(stmt)

In [ ]:
#stmt = games.update().values(isProcessed=False)
#conn.execute(stmt)

In [ ]:
conn.close()